# Build graph from preprocessed file
With a tabulated count:san file

## Read in packages, functions, and file

In [8]:
import sys
import networkx as nx
import matplotlib as plt
import numpy as np
import re
import chess
import subprocess
import timeit
import pickle
import stockfish
from stockfish import Stockfish
import chess
import chess.engine
import warnings
warnings.filterwarnings('ignore')

engine = chess.engine.SimpleEngine.popen_uci("../stockfish-10-64")
stockfish = Stockfish("../stockfish-10-64")

In [9]:
def varWeighted(scores, counts):
    if (len(counts)<2):
        return None
    else:
        weightedMean = sum([a*b for a,b in zip(scores,counts)])/sum(counts)
        scoreDiffs = [(score - weightedMean)**2 for score in scores]
        V1 = sum(counts)
        V2 = sum([count**2 for count in counts])
        var = (V1/(V1**2-V2)) * sum([a*b for a,b in zip(counts,scoreDiffs)])
        return var
    
def get_score(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    score = engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE)
    if('#' in str(score['score'])):
        if('-' in str(score['score'])):
            return -39765
        else:
            return 39765
    else:
        score = int(str(score["score"]))
    return score

def san_to_fen(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    return currFen

def get_node_sd(node):
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(node)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][node])
        return node, np.sqrt(varWeighted(scores,counts))
    except:
        return node, None

In [10]:
game_file = "../tab-trunc.txt"

with open(game_file, "r") as file_in:
    mainlineList = []
    for line in file_in:
        mainlineList.append(line)

#extract each string of moves
stringList = []
for mainLine in mainlineList:
    stringList.append(mainLine.split())

# stringList.sort(key = len) 


In [11]:
print(len(stringList))

692287


## Build and pickle graph

In [18]:
cnt = 0
start = timeit.default_timer()

#initialize graph with root node
g = nx.DiGraph()
g.add_node('root', score = 0, movelist = 'root')

for s in stringList[:1000]:
    whiteMove = 1
    if(cnt%10000 == 0):
        print("strings processed:", cnt, "TIME:", timeit.default_timer()-start)
    count = int(s[0])
    if(count >= 30):
        board = chess.Board()
        parentFen = 'root'
        for move in s[1:]:
            try:
                san = s[1:s.index(move)+1]
                board.push_san(move)
                currFen = board.fen()
                if(currFen not in g.nodes):
                    board = chess.Board(currFen)
                    if('#' not in move):
                        score = engine.analyse(board, chess.engine.Limit(time=0.05))
                        if('#' in str(score['score'])):
                            if('-' in str(score['score'])):
                                score =  -39765
                            else:
                                score =  39765
                        else:
                            score = int(str(score["score"])) * whiteMove
    #                         stockfish.set_fen_position(currFen)
    #                         score = stockfish.get_evaluation()['value']
                    else:
                        score = -39765 * whiteMove
                    g.add_node(currFen,
                               score = score,
                               count = count,
                               san = san,
                               movelistCount = {parentFen:count})
                    g.add_edge(parentFen, currFen)
                else:
                    if(parentFen in nx.get_node_attributes(g, 'movelistCount')[currFen]):
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] += count #increase parent count
                    else:
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count #add parent and count
                        g.add_edge(parentFen, currFen) #add edge from parentFen to currFen
                    g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count #increase count
    #                 whiteMove *= -1
                parentFen = currFen
            except Exception as e:
                print(e)
                print(sys.exc_info()[0])
                print("Move",s)
                stockfish.set_fen_position(currFen)
                print(stockfish.get_evaluation()['value'])
                break
        cnt+=1
    else:
        cnt+=1
            
stop = timeit.default_timer()
print('Time: ', stop - start)
print("totalNodes:",len(g))

strings processed: 0 TIME: 0.0006441919999815582
Time:  31.533026147999976
totalNodes: 654


## Once initial graph is built, calculate standard deviation and add as info to node

In [43]:
start = timeit.default_timer()
cnt = 0 
for node in g.nodes():
    if(cnt%10000 == 0):
        print("strings processed:", cnt, "TIME:", timeit.default_timer()-start)
    if node != 'root':
        scores = []
        counts = []
        for neighbor in list(g.neighbors(node)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'count')[neighbor])
        if(len(scores) < 2 or len(counts)<2):
            g.nodes[node]['sd'] = None
        else:
            g.nodes[node]['sd'] = np.sqrt(varWeighted(scores, counts))
        cnt +=1
stop = timeit.default_timer()
print('Time: ', stop - start)
print(g.nodes()['rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2'])  


strings processed: 0 TIME: 0.0002037059998656332
strings processed: 0 TIME: 0.0007345750000240514
16 [36, 72, 108, 95, 132, 100, 112, 128, 108, 118, 137, 109, 173, 113, 132, 153] 16 [2731, 4030, 1315, 821, 1052, 1933, 414, 491, 639, 262, 134, 34, 36, 57, 30, 36]
11 [112, 98, 40, 56, 165, 69, 354, 104, 104, 121, 166] 11 [1072, 105, 725, 1265, 112, 202, 30, 34, 42, 33, 84]
15 [79, 45, 26, 108, 131, 168, 78, 100, 182, 163, 215, 146, 114, 150, 113] 15 [1132, 505, 1398, 229, 91, 69, 108, 267, 31, 33, 31, 64, 43, 32, 35]
19 [98, 44, 77, 119, 23, 102, 140, 72, 139, 134, 107, 201, 188, 179, 116, 126, 151, 156, 176] 19 [4951, 7687, 4589, 2421, 21747, 5901, 2893, 10134, 4893, 2617, 780, 208, 194, 842, 386, 309, 193, 297, 42]
11 [-46, -4, 93, 5, 45, -2, 40, 56, 41, 53, 159] 11 [1772, 549, 1229, 587, 67, 187, 214, 137, 74, 44, 32]
14 [63, -18, -11, -23, 35, 37, 23, 49, 80, 42, 109, 61, 155, 110] 14 [1847, 1296, 2320, 480, 260, 776, 147, 347, 102, 41, 33, 36, 81, 32]
12 [31, -52, 32, 12, 52, 62, 13

5 [126, 159, 206, 195, 513] 5 [410, 78, 155, 79, 34]
22 [-16, 31, -33, -21, -13, 40, 44, 113, 66, 17, 86, 89, 11, 95, 48, 42, 25, 64, 93, 42, 73, 79] 22 [5817, 5501, 15508, 36599, 10410, 4294, 1197, 995, 1177, 2044, 1046, 1293, 341, 345, 198, 228, 172, 107, 65, 71, 144, 37]
9 [108, 90, 106, 59, 91, 23, 83, 141, 71] 9 [1478, 1321, 1157, 1142, 143, 205, 231, 109, 35]
10 [47, 139, 26, 89, 81, 150, 123, 91, 201, 236] 10 [1201, 1821, 1539, 315, 526, 105, 95, 66, 33, 41]
16 [70, 75, 57, 96, 70, 99, 198, 109, 202, 94, 99, 156, 226, 106, 220, 234] 16 [5823, 1029, 4247, 1537, 3902, 357, 351, 104, 164, 32, 32, 48, 31, 51, 36, 75]
12 [134, 64, 128, 116, 109, 69, 105, 74, 87, 136, 125, 146] 12 [2368, 10104, 3986, 13317, 2343, 3095, 532, 453, 208, 30, 32, 45]
5 [-72, 61, 57, 122, 124] 5 [1265, 489, 600, 176, 35]
2 [129, 139] 2 [1106, 159]
7 [-60, -20, 201, 90, -21, -22, -36] 7 [288, 465, 105, 33, 69, 32, 30]
2 [110, 139] 2 [244, 44]
2 [97, 154] 2 [406, 31]
2 [-62, -33] 2 [264, 142]
2 [-68, -58] 2 [

5 [862, -385, 49, 48, 40] 5 [801, 72, 65, 37, 30]
21 [151, 23, 122, 75, 75, 123, 146, 130, 136, 169, 224, 166, 151, 168, 103, 177, 189, 120, 175, 124, 145] 21 [1689, 14053, 937, 876, 2012, 1872, 326, 1066, 150, 441, 392, 457, 191, 32, 410, 233, 31, 126, 31, 49, 46]
8 [20, 0, -34, -27, -39, -89, -83, 11] 8 [158, 253, 279, 351, 371, 1903, 34, 32]
2 [865, 30] 2 [90, 68]
14 [42, -8, 25, 17, -3, 24, 61, -3, 20, 23, 50, 16, 83, 61] 14 [1687, 3216, 1769, 5528, 2313, 2110, 185, 361, 354, 184, 227, 103, 745, 72]
4 [895, 32, 34, 63] 4 [86, 726, 464, 255]
22 [63, 149, 107, 59, 131, 117, 175, 100, 125, 156, 110, 121, 81, 145, 221, 162, 322, 164, 143, 109, 224, 166] 22 [32142, 3077, 2154, 2105, 820, 2678, 447, 1920, 441, 486, 1227, 469, 458, 252, 160, 331, 36, 145, 58, 234, 32, 30]
22 [50, 61, 61, 109, 117, 80, 116, 83, 195, 93, 116, 84, 117, 73, 140, 171, 191, 87, 126, 162, 115, 160] 22 [12817, 892, 1731, 1189, 371, 991, 295, 406, 447, 1468, 197, 509, 575, 335, 67, 73, 74, 117, 36, 35, 37, 34]
27 

18 [105, 75, 63, 229, 92, 150, 183, 112, 187, 116, 223, 238, 288, 170, 168, 321, 156, 202] 18 [40599, 35743, 23917, 3438, 2127, 668, 738, 1100, 150, 184, 63, 198, 42, 627, 152, 49, 80, 33]
18 [-12, 57, 7, -29, -62, 81, 17, 67, 52, 54, -16, 54, 37, 115, -30, 4, 41, 42] 18 [9379, 19677, 3360, 4202, 5201, 65, 377, 320, 1041, 382, 303, 280, 72, 71, 111, 146, 38, 34]
10 [46, 83, 96, 16, 93, 89, 105, 130, 157, 190] 10 [4724, 890, 1837, 469, 340, 407, 454, 35, 90, 35]
9 [11, 171, 194, 128, 284, 212, 256, 254, 351] 9 [17963, 563, 893, 109, 33, 39, 32, 104, 35]
4 [60, 33, 124, 455] 4 [17631, 894, 111, 32]
13 [38, 17, 90, 141, 43, 69, 124, 78, 149, 216, 148, 98, 265] 13 [3347, 3855, 3480, 1762, 2337, 1603, 296, 477, 173, 134, 33, 34, 32]
3 [4, 48, 103] 3 [2492, 734, 35]
8 [65, 98, 70, 80, 137, 112, 145, 191] 8 [1071, 741, 456, 62, 30, 160, 39, 70]
6 [11, 92, 109, 40, 85, 53] 6 [439, 239, 111, 113, 70, 38]
2 [3, 20] 2 [338, 61]
2 [-5, 50] 2 [293, 45]
2 [50, -27] 2 [216, 77]
2 [5, 12] 2 [182, 70]


26 [158, 72, 50, 225, 162, 52, 109, 190, 107, 148, 115, 196, 106, 187, 194, 205, 142, 154, 174, 167, 171, 197, 148, 123, 161, 366] 26 [5862, 13833, 19416, 1129, 656, 1964, 8352, 1430, 1052, 541, 440, 801, 334, 466, 621, 169, 682, 473, 125, 165, 92, 37, 39, 35, 68, 32]
14 [122, 1, 256, -75, 85, 6, 12, 87, 51, 43, 116, 48, 0, 101] 14 [1192, 849, 361, 4464, 247, 805, 918, 95, 158, 116, 81, 31, 30, 43]
7 [39765, 39765, 95, -37, 227, 356, 59] 7 [238, 60, 435, 661, 123, 148, 70]
6 [39765, 39765, 39765, 781, 50, 16] 6 [145, 83, 48, 114, 650, 99]
2 [891, 938] 2 [44, 38]
17 [-14, 73, 78, 21, 59, -14, 66, 361, 77, 85, 22, 93, 613, 0, 130, 145, 39] 17 [69635, 3493, 8936, 1941, 596, 2356, 1253, 600, 222, 236, 74, 64, 69, 44, 40, 37, 32]
23 [205, 167, 69, 68, 110, 135, 216, 186, 227, 183, 175, 185, 148, 193, 182, 172, 183, 155, 164, 252, 157, 169, 240] 23 [3563, 18866, 2827, 25757, 4722, 11040, 182, 439, 1237, 1081, 640, 451, 399, 555, 213, 324, 121, 280, 242, 82, 31, 32, 30]
7 [-152, 539, -100, -7

...

3 [16, 15, -81] 3 [190, 62, 31]
2 [137, 124] 2 [68, 88]
2 [54, 171] 2 [182, 30]
4 [38, -64, -18, -16] 4 [31, 32, 32, 60]
2 [-30, -52] 2 [31, 71]
5 [-143, -136, -96, -100, -68] 5 [85, 120, 83, 69, 38]
2 [-108, -86] 2 [278, 34]
2 [91, 100] 2 [291, 40]
4 [0, 21, 25, 56] 4 [92, 30, 32, 31]
2 [69, 49] 2 [49, 43]
5 [-98, -113, -100, -29, -33] 5 [92, 41, 47, 30, 31]
2 [-26, -31] 2 [130, 104]
3 [98, 134, 84] 3 [36, 30, 30]
3 [38, 43, 46] 3 [115, 30, 67]
3 [-89, -72, 17] 3 [68, 34, 32]
7 [74, 33, 75, 82, 88, 47, 55] 7 [182, 152, 124, 106, 35, 63, 34]
3 [35, -51, 5] 3 [114, 115, 558]
3 [49, 166, 146] 3 [71, 31, 71]
2 [-24, 134] 2 [38, 75]
2 [-130, -148] 2 [110, 83]
2 [234, 268] 2 [86, 34]
5 [21, 17, 12, 22, 38] 5 [41, 38, 50, 30, 38]
3 [50, 56, 37] 3 [30, 65, 101]
2 [67, -2] 2 [70, 127]
2 [25, 66] 2 [37, 33]
2 [172, 169] 2 [42, 30]
4 [61, 52, 18, 48] 4 [80, 136, 472, 120]
2 [120, 45] 2 [33, 47]
4 [30, 52, 34, 66] 4 [79, 69, 42, 34]
3 [-76, -105, -126] 3 [86, 63, 36]
2 [-40, 149] 2 [123, 30]
3 [1

6 [-146, -103, -150, 44, -69, -4] 6 [196, 31, 32, 30, 32, 42]
3 [214, 207, 205] 3 [40, 114, 258]
2 [-47, -92] 2 [32, 38]
2 [37, 23] 2 [176, 135]
8 [-123, -65, -98, -125, -102, -42, 34, -57] 8 [195, 32, 31, 34, 32, 82, 37, 36]
4 [157, 171, 160, 207] 4 [31, 35, 33, 32]
3 [-15, 69, 36] 3 [299, 190, 36]
3 [37, 86, 58] 3 [69, 37, 47]
2 [273, -54] 2 [33, 31]
2 [71, 63] 2 [132, 31]
2 [26, 48] 2 [82, 99]
2 [2, -41] 2 [94, 36]
2 [104, 299] 2 [35, 31]
4 [38, 42, 25, 50] 4 [138, 100, 71, 49]
3 [69, 114, 100] 3 [43, 33, 31]
2 [211, 171] 2 [61, 37]
2 [100, 66] 2 [31, 30]
3 [48, 27, 87] 3 [132, 107, 31]
2 [21, 25] 2 [387, 33]
3 [5, 11, 67] 3 [34, 32, 141]
2 [0, 50] 2 [43, 738]
2 [7, -61] 2 [61, 105]
4 [-19, -57, -66, -46] 4 [160, 36, 398, 141]
3 [125, -111, -82] 3 [62, 33, 38]
2 [101, -30] 2 [157, 78]
3 [80, 55, 18] 3 [81, 92, 1501]
2 [62, -38] 2 [68, 32]
4 [26, 0, 73, 76] 4 [101, 1873, 844, 31]
2 [60, 10] 2 [155, 1326]
2 [-123, -36] 2 [77, 122]
6 [71, 75, -25, 12, 51, 2] 6 [104, 43, 149, 392, 98, 3

2 [7, 127] 2 [36, 460]
2 [53, 23] 2 [62, 44]
2 [-123, -74] 2 [75, 74]
2 [75, 81] 2 [44, 39]
2 [67, 109] 2 [37, 140]
2 [112, 146] 2 [46, 200]
4 [-165, -149, -145, -184] 4 [37, 34, 48, 142]
2 [6, -5] 2 [32, 31]
2 [10, 19] 2 [93, 33]
2 [89, 2] 2 [30, 105]
5 [98, 37, 43, 51, 145] 5 [32, 61, 84, 86, 74]
3 [54, 82, 98] 3 [132, 106, 33]
2 [-133, -149] 2 [77, 42]
2 [12, 75] 2 [69, 32]
2 [132, 189] 2 [69, 35]
2 [73, 28] 2 [32, 33]
3 [-113, -107, -115] 3 [70, 105, 109]
3 [60, 75, 95] 3 [38, 84, 43]
3 [5, 50, 32] 3 [33, 270, 34]
3 [-130, -136, -65] 3 [30, 95, 30]
5 [-37, 45, 45, 62, 72] 5 [76, 37, 34, 39, 32]
3 [81, 70, 136] 3 [34, 47, 40]
3 [3, -80, -7] 3 [32, 38, 76]
2 [-5, 57] 2 [312, 187]
5 [18, -14, 3, 64, 37] 5 [1047, 237, 36, 32, 270]
3 [49, 7, 39] 3 [81, 74, 43]
2 [72, 86] 2 [32, 47]
2 [55, 21] 2 [34, 37]
3 [35, 34, -4] 3 [114, 628, 2113]
2 [111, 133] 2 [34, 32]
2 [-45, -12] 2 [38, 36]
3 [31, 14, 14] 3 [30, 1193, 39]
2 [165, 159] 2 [66, 46]
5 [172, 124, 145, 168, 160] 5 [110, 1096, 296, 3

3 [-63, -128, -115] 3 [32, 31, 55]
3 [51, 44, 61] 3 [76, 222, 489]
2 [-55, 42] 2 [82, 52]
2 [-420, -54] 2 [32, 31]
3 [23, 48, 13] 3 [135, 30, 31]
8 [63, -1, -47, 41, -26, -26, -56, 15] 8 [82, 30, 47, 35, 33, 30, 48, 37]
2 [76, -1] 2 [2104, 135]
2 [33, 71] 2 [37, 30]
7 [4, 70, -57, 22, 50, -51, 22] 7 [39, 123, 163, 31, 34, 142, 74]
11 [-61, -58, -115, -57, -27, -10, -21, -47, -39, -17, -64] 11 [291, 352, 1195, 114, 39, 34, 34, 41, 34, 32, 43]
4 [139, 204, 197, 177] 4 [33, 68, 72, 36]
2 [-25, -27] 2 [96, 31]
4 [52, 37, -10, 16] 4 [429, 30, 37, 39]
2 [-42, -50] 2 [71, 30]
2 [14, 42] 2 [1198, 247]
5 [161, 101, 103, 157, 170] 5 [118, 182, 31, 34, 32]
3 [-44, -14, -58] 3 [36, 50, 72]
2 [136, 105] 2 [46, 37]
4 [-1, -37, 29, -9] 4 [45, 37, 36, 40]
3 [-87, -96, -51] 3 [36, 32, 34]
3 [63, 57, 31] 3 [45, 30, 37]
2 [72, 129] 2 [53, 35]
2 [166, 195] 2 [258, 32]
5 [-98, -96, 61, 17, 4] 5 [77, 30, 30, 54, 33]
2 [50, 98] 2 [32, 35]
5 [-37, -33, -86, -24, 15] 5 [413, 85, 62, 32, 50]
3 [166, 185, 118] 3

2 [-75, -51] 2 [88, 30]
3 [165, 190, 137] 3 [64, 34, 38]
2 [28, 86] 2 [164, 56]
2 [-62, -86] 2 [236, 76]
3 [-63, -87, -102] 3 [31, 59, 37]
9 [-68, -12, -35, -75, 44, -71, -13, 0, -4] 9 [195, 43, 39, 50, 51, 77, 53, 36, 34]
5 [-210, -122, -151, -139, -164] 5 [126, 141, 831, 43, 84]
2 [-15, 19] 2 [32, 34]
2 [73, 17] 2 [35, 156]
2 [-18, -75] 2 [31, 30]
2 [0, -113] 2 [36, 47]
2 [-18, 31] 2 [75, 196]
2 [87, 83] 2 [31, 33]
2 [48, -23] 2 [36, 92]
3 [-158, -119, -143] 3 [73, 100, 291]
2 [8, -31] 2 [70, 37]
2 [-34, -19] 2 [244, 30]
2 [-87, 384] 2 [78, 31]
2 [71, 38] 2 [30, 30]
2 [-109, -96] 2 [38, 31]
2 [-57, 5] 2 [180, 32]
2 [-111, -86] 2 [33, 32]
2 [29, -62] 2 [40, 43]
4 [152, 170, 83, 156] 4 [42, 93, 36, 36]
2 [-60, -87] 2 [59, 43]
7 [12, 35, -28, -21, 10, -25, 29] 7 [123, 39, 52, 67, 94, 74, 33]
3 [50, 14, 5] 3 [36, 1198, 37]
2 [49, 44] 2 [65, 34]
2 [-138, -27] 2 [1134, 30]
2 [-11, -40] 2 [63, 110]
6 [34, 76, 143, 50, 37, 127] 6 [76, 266, 32, 667, 66, 32]
2 [-121, -156] 2 [32, 32]
2 [25, 46

In [44]:
#nx.write_gpickle(g,"test.gpickle")

In [46]:
g = nx.read_gpickle('../graph-pickle/690k_30_sd.gpickle')

In [47]:
g.nodes()[san_to_fen(['c4', 'Nf6', 'Nf3'])]

{'score': 7,
 'count': 430,
 'san': ['c4', 'Nf6', 'Nf3'],
 'movelistCount': {'rnbqkb1r/pppppppp/5n2/8/2P5/8/PP1PPPPP/RNBQKBNR w KQkq - 1 2': 430},
 'sd': 9.61816517789542}

## Test variance calcuilations

In [11]:
start = timeit.default_timer()

board = chess.Board()
board.push_san('d4') #push the move to the board
board.push_san('Nf6') #push the move to the board
currFen = board.fen()
print(currFen)
scores = []
counts = []
for neighbor in list(g.neighbors(currFen)):
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'count')[neighbor])
    
print("sd:",np.sqrt(varWeighted(scores, counts)))

stop = timeit.default_timer()
print('Time: ', stop - start) 

rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
sd: 36.453967228234106
Time:  2.1070754819957074


In [12]:
gamesDict = {"root" : ['e4', 'c5'],
"safe" : ['e4', 'c5', 'c3'],
"risky" : ['e4', 'c5', 'd4'],
"root2" : ['e4', 'e6', 'd4', 'd5'],
"popular2.0" : ['e4', 'e6', 'd4', 'd5', 'Nc3'],
"popular2.1" : ['e4', 'e6', 'd4', 'd5', 'Nd2'],
"safe2" : ['e4', 'e6', 'd4', 'd5', 'exd5'],
"risky2" : ['e4', 'e6', 'd4', 'd5', 'e5'],
"Ruy" : ['e4', 'e5','Nf3', 'Nc6','Bb5'],
"Vienna" : ['e4' ,'e5','d4','exd4','c3']}

for key, game in gamesDict.items():
    currFen = san_to_fen(game)
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(currFen)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
        print('{} {} ||| SD :'.format(key, game), np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))
    except:
        break

    

root ['e4', 'c5'] ||| SD : 53.17421298817888 , total Games: 214412
safe ['e4', 'c5', 'c3'] ||| SD : 33.803211905697694 , total Games: 9288
risky ['e4', 'c5', 'd4'] ||| SD : 112.59554494204002 , total Games: 19407
root2 ['e4', 'e6', 'd4', 'd5'] ||| SD : 35.29555838233849 , total Games: 42146
popular2.0 ['e4', 'e6', 'd4', 'd5', 'Nc3'] ||| SD : 38.82435952658399 , total Games: 8875
popular2.1 ['e4', 'e6', 'd4', 'd5', 'Nd2'] ||| SD : 73.60617708334175 , total Games: 3144
safe2 ['e4', 'e6', 'd4', 'd5', 'exd5'] ||| SD : 59.07250112917537 , total Games: 11806
risky2 ['e4', 'e6', 'd4', 'd5', 'e5'] ||| SD : 78.55239442769992 , total Games: 20829
Ruy ['e4', 'e5', 'Nf3', 'Nc6', 'Bb5'] ||| SD : 19.630732457868167 , total Games: 42312
Vienna ['e4', 'e5', 'd4', 'exd4', 'c3'] ||| SD : 35.14133659917036 , total Games: 6424


In [13]:
sd_dict = {}
for node in g.nodes():
    n, val = get_node_sd(node)
    sd_dict.update({n: val})
#     dict.update(Iterable_Sequence of key:value)
#     sd_list.append(get_node_sd(node))

sd_dict = {key:val for key, val in sd_dict.items() if val != None}

maxSD = max(sd_dict.items(), key=lambda x : x[1])
print('Max value in Dict: ', maxSD[1])
print('Key With Max value in Dict: ', maxSD[0])

Max value in Dict:  28366.29563407954
Key With Max value in Dict:  r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4


In [14]:
{k: v for k, v in sorted(sd_dict.items(), key=lambda item: item[1], reverse = True)}

{'r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4': 28366.29563407954,
 'rnb1k1nr/pppp2pp/8/4NP2/7q/8/PPPP1KPP/RNBQ1B1R w kq - 1 6': 28305.484450897497,
 'rnbqkb1r/ppp2ppp/3p1n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 2 4': 28248.208801621386,
 'rnb1k1nr/pppp1ppp/8/2b5/2B1Pp1q/8/PPPP2PP/RNBQ1KNR w kq - 4 5': 28188.10472522053,
 'rnb1k1nr/pppp1ppp/5q2/2b1p3/8/1P4P1/P1PPPPBP/RNBQK1NR w KQkq - 1 4': 28142.142784443404,
 'rnbqkbnr/pp3ppp/2pp4/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 1 4': 28130.82907594442,
 'rn1qkbnr/pppb1ppp/3p4/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 3 4': 28078.503174136615,
 'r1bqkb1r/ppp3pp/2n2p2/3Bp1N1/8/5Q2/PPPP1PPP/RNB1K2R b KQkq - 0 7': 28053.754436795083,
 'r1b1k2r/pppp1Npp/2n5/4p3/2B1n2q/8/PPPP2PP/RNBQ2KR w kq - 2 8': 28047.39047576441,
 'r1bqkbnr/pp1p1pp1/7p/2pQ4/2B1P3/8/PPP2PPP/RNB1K2R b KQkq - 1 7': 27987.28639936355,
 'rnbqkbnr/pppp1ppp/8/4p3/6P1/5P2/PPPPP2P/RNBQKBNR b KQkq - 0 2': 27879.099061842007,
 'rn1qkbnr/ppp2ppp/3p4/4p3/2BPP3/5Q2/

In [16]:
currFen =  'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6'
scores = []
counts = []

for neighbor in list(g.neighbors(currFen)):
    print(neighbor, g.nodes()[neighbor])
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
print('SD :', np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))


rnb1kbnr/p3pppp/8/qp6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 2150, 'count': 113, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Qa5+'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 113}}
r1bqkbnr/p3pppp/2n5/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 462, 'count': 190, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Nc6'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 190}}
rnb1kbnr/p1q1pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 420, 'count': 34, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Qc7'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 34}}
rn1qkbnr/pb2pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 655, 'count': 63, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Bb7'], 'movelistCount':

## Ad hoc viz of moves and children with sf score

In [ ]:
#Move ['32', 'f3', 'e5', 'g4', 'Qh4#']

board = chess.Board()
board.push_san('f3') #push the move to the board
board.push_san('e5') #push the move to the board
#board.push_san('g4') #push the move to the board
#board.push_san('Qh4#') #push the move to the board
currFen = board.fen()
print(currFen)
stockfish.set_fen_position(currFen)
score = stockfish.get_evaluation()['value']
print(score)

In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
board

In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
print("CURRFEN:")
print(board)
print("_"*50)
print("_"*50)
for neighbor in list(g.neighbors(currFen)):
    board = chess.Board(neighbor)
    print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
    print(board)
    print("_"*50)